In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', None)
sugarcane = pd.read_csv('datos_tesis.csv')
sugarcane_copy = sugarcane.copy()
# PerfilDatos = data.profile_report(title = "Reporte Datos Tesis", plot = {'histogram': {'bins' : 20}})
# PerfilDatos.to_file("SugarcaneReport.html")

C:\ProgramData\Anaconda3\envs\models_tesis\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## DATA QUALITY 
### ABOUT MODIS

REVIEW DOI IN README FILE
![title](1.png)

In [2]:
sugarcane.MOD13Q1_006__250m_16_days_pixel_reliability.unique()
cloud_NDVI = (sugarcane.MOD13Q1_006__250m_16_days_pixel_reliability > 1)
cloud_NDVI_sum = cloud_NDVI.sum()
# RAW NDVI DOWNLOAD REPRESENTS WHICH PERCENT?
cloud_NDVI_sum/len(sugarcane)
# Removing samples with missing values comes with *a cost of losing probable valuable data and the associated information or patterns*. A better strategy is to impute the missing values. Here, i'll impute the missing value in the dataset by the ‘mean’ for NDVI. 
useful_NDVI_mean = (sugarcane.MOD13Q1_006__250m_16_days_pixel_reliability <= 1).mean()
sugarcane_copy.MOD13Q1_006__250m_16_days_NDVI[cloud_NDVI] = useful_NDVI_mean
sugarcane_copy.MOD13Q1_006__250m_16_days_NDVI.mean()
sugarcane.MOD13Q1_006__250m_16_days_NDVI.mean()
#Now, delete columns with descriptions about the NDVI parameter.
sugarcane_copy.drop(['index_y', 'MODIS_Tile','MOD13Q1_006__250m_16_days_composite_day_of_the_year',
       'MOD13Q1_006__250m_16_days_VI_Quality',
       'MOD13Q1_006__250m_16_days_pixel_reliability',
       'MOD13Q1_006__250m_16_days_VI_Quality_VI_Usefulness_Description',
       'MOD13Q1_006__250m_16_days_pixel_reliability_MODLAND_Description',
       'header.end'], axis=1, inplace=True)

## DUPLICATE ROWS
sugarcane_copy.drop_duplicates(keep = "first", inplace = True, ignore_index = True)
sugarcane_1 = sugarcane_copy # Backup
## CHANGING DATA TYPES
numeric_features = list(sugarcane_copy.describe().columns.values)
all_columns = list(sugarcane_copy.columns.values)
categorical_features = set(all_columns) - set(numeric_features)
categorical_features
sugarcane_copy.edad.unique()
# Regex substitution for 'Edad' feature; there are values with comma for decimal notation and others with dot as decimal notation
sugarcane_copy['edad'].replace(to_replace='[^,0-9.]+', value='',inplace=True,regex=True)
sugarcane_copy['edad'].replace(to_replace='[,]+', value='.',inplace=True,regex=True)
sugarcane_copy['edad']=pd.to_numeric(sugarcane_copy['edad'])
sugarcane_copy['edad'].replace(0, round(sugarcane_copy['edad'].mean(),2),inplace=True)

# Regex substitution for 'Area' feature; there are values with comma for decimal notation and others with dot as decimal notation and other special characters
sugarcane_copy.area.unique()
sugarcane_copy['area'].replace(to_replace='[^0-9.,]+', value='',inplace=True,regex=True)
sugarcane_copy['area'].replace(to_replace='[,]+', value='.',inplace=True,regex=True)
sugarcane_copy['area'].replace('.', 0,inplace=True)
sugarcane_copy['area']=pd.to_numeric(sugarcane_copy['area'])
sugarcane_copy['area'].replace(0, round(sugarcane_copy['edad'].mean(),2),inplace=True)

# Regex substitution for 'corte' feature with mode; there are values with dot and 'C' characters
# percent outliers in 'corte'
(len(sugarcane_copy[sugarcane_copy['corte'] == '.']) + len(sugarcane_copy[sugarcane_copy['corte'] == 'c'])
     + len(sugarcane_copy[sugarcane_copy['corte'] == 'c ']) + len(sugarcane_copy[sugarcane_copy['corte'] == 'C '])
     )/len(sugarcane_copy['corte'])

sugarcane_copy['corte'].replace('.', sugarcane_copy['corte'].mode()[0], inplace = True)
sugarcane_copy['corte'].replace('c', sugarcane_copy['corte'].mode()[0], inplace = True)
sugarcane_copy['corte'].replace('c ', sugarcane_copy['corte'].mode()[0], inplace = True)
sugarcane_copy['corte'].replace('C ', sugarcane_copy['corte'].mode()[0], inplace = True)

# And now . .  .
numeric_features_1 = list(sugarcane_copy.describe().columns.values)
all_columns_1 = list(sugarcane_copy.columns.values)
categorical_features_1 = set(all_columns_1) - set(numeric_features_1)

sugarcane_copy['variedad'].replace('MISCELANEAS', 'MISCELANEA', inplace = True)
sugarcane_copy['variedad'] = sugarcane_copy['variedad'].astype("category")
sugarcane_copy['corte'] = sugarcane_copy['corte'].astype("category")
sugarcane_copy['hacienda'] = sugarcane_copy['hacienda'].astype("category")
sugarcane_copy['lote_comercial'] = sugarcane_copy['lote_comercial'].astype("category")
sugarcane_copy['lote_semillero'] = sugarcane_copy['lote_semillero'].astype("category")
sugarcane_copy['Date'] = sugarcane_copy['Date'].astype('datetime64[ns]')
sugarcane_copy['year'] = sugarcane_copy['Date'].dt.year
sugarcane_copy['month'] = sugarcane_copy['Date'].dt.month   
sugarcane_copy.rename(columns = {'x':'longitude', 'y':'latitude'}, inplace=True)

## 'RSD' AND 'LSD'
## Regex substitution for 'RSD' and 'LSD' features

sugarcane_copy['RSD'].replace(to_replace='0',value=0, inplace=True)
sugarcane_copy['RSD'].replace(to_replace='.',value=0, inplace=True)
sugarcane_copy['RSD'].replace(to_replace='5',value=5, inplace=True)
sugarcane_copy['RSD'].replace(to_replace='15',value=15, inplace=True)
sugarcane_copy['RSD'].replace(to_replace='10',value=10, inplace=True)

sugarcane_copy['LSD'].replace(to_replace='0',value=0, inplace=True)
sugarcane_copy['LSD'].replace(to_replace='.',value=0, inplace=True)
sugarcane_copy['LSD'].replace(to_replace='5',value=5, inplace=True)
sugarcane_copy['LSD'].replace(to_replace='5.2',value=5.2, inplace=True)
sugarcane_copy['LSD'].replace(to_replace='10',value=10, inplace=True)

## MISSING VALUE IMPUTATION
numeric_features3 = list(sugarcane_copy.describe().columns.values)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(sugarcane_copy[numeric_features3])
sugarcane_copy[numeric_features3] = imputer.transform(sugarcane_copy[numeric_features3])
sugarcane_copy=sugarcane_copy.round(2)
sugarcane_copy[['ingenio','year','month']]=sugarcane_copy[['ingenio','year','month']].astype(int)
sugarcane_copy.describe().T


C:\ProgramData\Anaconda3\envs\models_tesis\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,count,mean,std,min,25%,50%,75%,max
ingenio,20805.0,9.476520,2.956372,1.00,7.00,10.00,11.00,13.00
area,20805.0,2.883278,2.420547,0.01,1.58,2.66,3.81,81.00
edad,20805.0,7.029627,0.845867,0.91,6.50,7.00,7.40,14.40
RSD,20805.0,0.204999,1.148691,0.00,0.00,0.00,0.00,15.00
LSD,20805.0,0.183975,1.001665,0.00,0.00,0.00,0.00,10.00
...,...,...,...,...,...,...,...,...
T2M,20805.0,17.200622,2.046093,13.37,15.83,16.58,18.23,24.03
T2M_MAX,20805.0,22.050204,2.233425,15.99,20.48,21.78,23.24,30.37
T2M_MIN,20805.0,13.585787,2.315754,7.58,12.04,13.08,14.72,20.75
year,20805.0,2018.149964,1.267373,2007.00,2017.00,2018.00,2019.00,2020.00


# Quantile-based Flooring and Capping

In [3]:
print(sugarcane_copy['area'].quantile(0.1))
print(sugarcane_copy['area'].quantile(0.9))

0.68
4.87


In [4]:
sugarcane_copy['area'] = np.where(sugarcane_copy['area'] < sugarcane_copy['area'].quantile(0.1), sugarcane_copy['area'].quantile(0.1),sugarcane_copy['area'])
sugarcane_copy['area'] = np.where(sugarcane_copy['area'] > sugarcane_copy['area'].quantile(0.9), sugarcane_copy['area'].quantile(0.9),sugarcane_copy['area'])
print(sugarcane_copy['area'].skew())

0.11358725092029193


In [6]:
sugarcane_copy.to_csv('sugarcane.csv', index=False)